## Robot setup

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("..")
sys.path.append("../../")

from armControl import *
from forwardKinematics import robot_config
from utils.motor_setup import maxonGearSmall
import armControl

import signal

socket_ip = '192.168.0.104'
socket_port = 1125

#trajPlanner = trajectoryGenerator()

motors = maxonGearSmall()
motors.tcp_init(socket_ip, socket_port)

#axis_motor_indexes = np.array([-1, -1, -1, 0, 3, 2, 1]) #indexed from 0 for motors and axis
#velocity = np.ones(8)*3.14/20


Opening socket at ip: 192.168.0.104 using port: 1125


## Command motors w/ indexing for joints

def commandJoints(setpoint_arm, trajectory = True):
    myArm = armControl.remoteRobotArm()
    myArm.jointAngleSetpoint = setpoint_arm
    myArm.updateMotorArmMixing()
    setpoint_motor = np.zeros(8)
    setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
    if trajectory:
        motors.run_trajectory(setpoint_motor, velocity)
    else:
        motors.command_motors_radians(setpoint_motor)


## Arm Motors

In [10]:
print("Arming motors now...")
motors.arm_motors()

Arming motors now...
initializing motors to [  2649  45421    414 -14617  -1193   1210   1796      0]
*** C side has an error or needs to be armed ***

Arming motors


## Manual motor zeroing

In [12]:
from importlib import reload
reload(armControl)

myArm = armControl.remoteRobotArm()

setpoint_arm = np.array([0.00,0.0,-0.0,0.06,0,-0.3,0])
myArm.commandJoints(motors, setpoint_arm)
time.sleep(0.5)


In [13]:
for i in range(8):
    motors.zero_motors_radians(i, motors.get_motors_position_radians()[i])

## Test arm joint mixing

### this is old, now in "commandJoints" above

import armControl
from importlib import reload
reload(armControl)

setpoint_arm = np.array([0,0,0,0,0,0,-0])
myArm = armControl.remoteRobotArm()
myArm.jointAngleSetpoint = setpoint_arm
myArm.updateMotorArmMixing()
print(myArm.motorAngleSetpoint)

setpoint_motor = np.zeros(8)
setpoint_motor[axis_motor_indexes[3:7]] = myArm.motorAngleSetpoint[3:7]
print(setpoint_motor)
motors.run_trajectory(setpoint_motor, velocity)


In [ ]:
print(myArm.jointUpperLimits)
print(myArm.jointLowerLimits)
a = np.linspace(myArm.jointLowerLimits[-1],myArm.jointUpperLimits[-1], num=10)
b = np.linspace(myArm.jointLowerLimits[-2],myArm.jointUpperLimits[-2], num=10)
c = np.linspace(myArm.jointLowerLimits[-3],myArm.jointUpperLimits[-3], num=10)
d = np.linspace(myArm.jointLowerLimits[-4],myArm.jointUpperLimits[-4], num=10)

print(a)
print(b)
#c = np.meshgrid(a,b)
#print(c)

setpoint_arm = np.array([0,0,0,0.,0,0.,0.0])
myArm.commandJoints(motors, setpoint_arm)

time.sleep(2.5)

for k in range(len(d)):
    #time.sleep(5)
    for j in range(len(c)):
        #time.sleep(5)
        for i in range(len(b)):
            setpoint_arm = np.array([0,0,0,d[k],c[j],b[i],0])
            myArm.commandJoints(motors, setpoint_arm, trajectory = True)
            time.sleep(0.5)
        time.sleep(2)


setpoint_arm = np.array([0,0,0,0.,0,0.,0.0])
myArm.commandJoints(motors, setpoint_arm)

[8.25e+01 4.25e+01 1.75e+00 8.72e-01 8.72e-01 8.72e-01 4.75e-02]
[-82.5   -42.5    -1.75   -0.872  -0.872  -0.872   0.   ]
[0.         0.00527778 0.01055556 0.01583333 0.02111111 0.02638889
 0.03166667 0.03694444 0.04222222 0.0475    ]
[-0.872      -0.67822222 -0.48444444 -0.29066667 -0.09688889  0.09688889
  0.29066667  0.48444444  0.67822222  0.872     ]


## Sine wave / profile test

In [196]:
dt = 0.005
runtime = 10
num_elements = int(runtime/dt)
setpoints = np.zeros((num_elements, 7))
frequency = 0.1
amplitude = np.pi/6
for i in range(num_elements):
    setpoints[i,:] = (np.ones(7) * np.sin(i*dt*2*pi*frequency)*amplitude)


start_time = time.time()

currents_mA = []
for i in range(num_elements):
    current_time = time.time()
    setpoint = setpoints[i]
    myArm.commandJoints(motors, setpoint, trajectory=False)
    time.sleep(dt)

time.sleep(1.5)

motors.run_trajectory(np.zeros(8), velocity)


OSError: [Errno 9] Bad file descriptor

## Cleanup

In [8]:
setpoints = np.zeros(8)
velocity = np.ones(8)*3.14/5
motors.run_trajectory(setpoints, velocity)
time.sleep(3)
motors.tcp_close()

In [11]:
motors.tcp_close()